In [ ]:
import pySPM
import numpy as np
import matplotlib.pyplot as plt
import peakutils as pk
from scipy.signal import savgol_filter

In [ ]:
afmdir = '/users/zatterin/Documents/data/AFM/kto_pto/'
d = 'EDOARDO/29.03.18/day4/'

In [ ]:
ls {afmdir+d}

In [ ]:
fname = afmdir+d+'04012029.0_00001.spm'

In [ ]:
print(fname)

In [ ]:
a = pySPM.Bruker(fname)
a.list_channels()

In [ ]:
la = a.get_channel(channel='Amplitude2',backward=True)

In [ ]:
la.size

In [ ]:
class LineProjection(object):
    def __init__(self, img, ax, i, q1, q2, n):
        self.img = img
        self.q1 = q1
        self.q2 = q2
        self.data = i[:-1,:-1]#np.array(img.get_array()).reshape(img._meshHeight,img._meshWidth)
        self.cidclick = img.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.cidrelease = img.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.ax = ax
        self.n = n
        
    def on_press(self,event):
        self.press = event.xdata, event.ydata    
        if event.inaxes != self.img.axes: return
        print('pressed: {0}'.format(self.press))
        
    def on_release(self,event):
        global x, y, Icut, box, mask, maskimg
        
        start_q1, start_q2 = self.press
        end_q1, end_q2 = event.xdata, event.ydata
        print('released: {0}, {1}'.format(end_q1, end_q2))

        # Make array of qx, qy values along line        
        x = np.linspace(start_q1,end_q1,self.n) 
        y = np.linspace(start_q2,end_q2,self.n)
        
        # Calc
        ind = np.sqrt((q2[:-1,:-1].ravel() - y[:,None])**2 + (q1[:-1,:-1].ravel() - x[:,None])**2) < (10) #min along column
        Icut = ((self.data.ravel() * np.ones(ind.shape[0])[:,None]) * ind).sum(1)
        box = ind.max(0).reshape(self.data.shape)
        mask = np.ma.masked_where(box==1, box)
        
        # Plot img and cut line - ax0
        if len(self.img.axes.lines) == 0:
            self.img.axes.plot([start_q1, end_q1], [start_q2, end_q2])
        else:
            self.img.axes.lines[0].set_data([start_q1, end_q1], [start_q2, end_q2])
        self.img.figure.canvas.draw_idle()
            
        # Show integrated area - ax1
        self.ax[1].clear()
        ax[1].pcolormesh(q1,q2,i)
        ax[1].pcolormesh(q1,q2,mask,alpha=0.2)
        self.ax[1].set_aspect('equal')
        self.ax[1].figure.canvas.draw_idle()
            
        # Draw Lines - ax2
        self.ax[2].clear()
#         self.ax[2].plot((np.sqrt(x**2+y**2)),Icut)
        self.ax[2].plot(Icut)
        self.ax[2].set_xlabel('Q')
        self.ax[2].set_yscale('log')
        self.ax[2].figure.canvas.draw_idle()

In [ ]:
%matplotlib qt5

fig, ax = plt.subplots(1,3,figsize=(10,5))

grid = np.linspace(0,500,256)
q1, q2 = np.meshgrid(grid,grid)
i = la.pixels

img = ax[0].pcolormesh(q1,q2,i)
ax[0].set_aspect('equal')
test = LineProjection(img,ax,i,q1,q2,200)

In [ ]:
# %matplotlib inline

# Select cut coordinates
start_q1, start_q2 = 232.20766129032256, 247.0352822580645
end_q1, end_q2 = 361.59274193548384, 129.64616935483872

# Select cut coordinates #2
start_q11, start_q22 = 63.4072580645161, 170.77520161290323
end_q11, end_q22 = 228.78024193548384, 360.14012096774195

# Make array of qx, qy values along line        
x = np.linspace(start_q1,end_q1,200) 
y = np.linspace(start_q2,end_q2,200)

# Make array of qx, qy values along line #2       
xx = np.linspace(start_q11,end_q11,200) 
yy = np.linspace(start_q22,end_q22,200)

# Data
grid = np.linspace(0,500,256)
q1, q2 = np.meshgrid(grid,grid)
I = la.pixels[:-1,:-1]
# i += 1

# Calc
ind = np.sqrt((q2[:-1,:-1].ravel() - y[:,None])**2 + (q1[:-1,:-1].ravel() - x[:,None])**2) < (10) #min along column
Icut_xy = ((I.ravel() * np.ones(ind.shape[0])[:,None]) * ind).sum(1)
box = ind.max(0).reshape(I.shape)
mask = np.ma.masked_where(box==1, box)
# Calc #2
ind2 = np.sqrt((q2[:-1,:-1].ravel() - yy[:,None])**2 + (q1[:-1,:-1].ravel() - xx[:,None])**2) < (10) #min along column
Icut_xy2 = ((I.ravel() * np.ones(ind2.shape[0])[:,None]) * ind2).sum(1)
box2 = ind2.max(0).reshape(I.shape)
mask2 = np.ma.masked_where(box2==1, box2)

# Plot
fig, ax = plt.subplots(1,2,figsize=(12, 4),dpi=100)

ax[0].pcolormesh(q1,q2,np.log(I))
ax[0].pcolormesh(q1,q2,mask,alpha=0.2)
ax[0].pcolormesh(q1,q2,mask2,alpha=0.2)
ax[0].set_title('Box being integrated')
ax[0].set_aspect('equal')
ax[0].set_xlabel('Qx')
ax[0].set_ylabel('Qy')

ax[1].plot((np.sqrt((x-x.min())**2+(y-y.max())**2)),Icut_xy)
ax[1].plot((np.sqrt((xx-xx.min())**2+(yy-yy.min())**2)),Icut_xy2)
ax[1].set_title('Integrated profile')
ax[1].set_xlabel('Distance [nm]')

In [ ]:
_103i = Icut_xy2

In [ ]:
%matplotlib inline
## Calc
Icut_xy = savgol_filter(_103i,5,0)
idx_xy = pk.indexes(Icut_xy,thres=0.1,min_dist=10)
q = np.sqrt((x-x.min())**2+(y-y.max())**2)
# The periodicity - calc
order = np.array(range(len(idx_xy)))
m,b = np.polyfit(order,q[idx_xy],1)
period = m

## plot
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(8,4))
plt.suptitle('e17089 - lat pfm cut2 - period: {0} nm'.format(np.round(period,decimals=2)))

# The detected peaks
ax1.plot(q,Icut_xy)
ax1.scatter(q[idx_xy],Icut_xy[idx_xy],c='r')
ax1.set_xlabel('Q')

# The fit
ax2.scatter(order,q[idx_xy],c='r')
ax2.plot(order,order*m+b)
plt.show()

# Output
print('Period: {0} nm'.format(period))